In [ ]:
from IPython.display import display, HTML
from ipywidgets import widgets, Layout, GridspecLayout, AppLayout

import ipyaggrid
from ipyaggrid import Grid

In [ ]:
import numpy as np
import pandas as pd
import json
import requests

from ipyleaflet import (Map, basemaps, basemap_to_tiles, ImageOverlay, WMSLayer,
                        projections, GeoData, LayerGroup, LayersControl, WidgetControl)

In [ ]:
rid = 20

try:
    extent = requests.get(url=f"http://localhost:9090/api/v1/hazard/raster/extent/{rid}")
except requests.ConnectionError:
    display("Initial API call failed - is the API up?")
    # exit

if extent.status_code == 500:
    # try again
    extent = requests.get(url=f"http://localhost:9090/api/v1/hazard/raster/extent/{rid}")

bounds = json.loads(extent.json())

In [ ]:
m = Map(center=(-5,40), zoom=4, 
        # basemap=basemaps.Gaode.Satellite)
        basemap=basemaps.Stamen.Watercolor)

# control = LayersControl(position='topright')
# m.add_control(control)

In [ ]:
api_img = ImageOverlay(
    name='api_test',
    url=f'http://localhost:9090/api/v1/hazard/raster/{rid}',
    bounds=bounds,
    opacity=0.9
)

m.add_layer(api_img)

In [ ]:
# Add custom controls via WidgetControl?
# https://ipyleaflet.readthedocs.io/en/latest/api_reference/widget_control.html

# Or display DataFrame as table 
# https://stackoverflow.com/a/29665452/2694952

# Need some way of linking an "on-click" or "on-select" event on the table to the map...
# https://medium.com/swlh/how-to-use-mouse-events-on-ipyleaflet-4d002097efc0

# On select event for aggrid
# https://ag-grid.com/documentation/javascript/row-selection/#selection-events

# Ipyvuetify might be an alternative...
# https://github.com/mariobuikhuizen/ipyvuetify/issues/71#issue-630953624
# https://github.com/mariobuikhuizen/ipyvuetify
# jupyter labextension install jupyter-vuetify

# Cache API calls
# https://technology.amis.nl/data-analytics/jupyter-notebook-for-retrieving-json-data-from-rest-apis/

# Useful Aggrid resources
# https://dgothrek.gitlab.io/ipyaggrid/guide/customize.html#custom-buttons
# https://hub.gke2.mybinder.org/user/dgothrek-ipyaggrid-hx37qvp0/notebooks/demo-ipyaggrid-customize.ipynb#Building-custom-buttons
# 

In [ ]:
df = pd.DataFrame({
    'Name': ['Data name', 'Some other name'],
    'Info': ['some metadata', 'some other metadata'],
    '-': ['Download' for i in range(2)]
})


data_list = requests.get(url=f"http://localhost:9090/api/v1/poc/available-data")
if data_list.status_code == 200:
    available_rasters = pd.read_json(data_list.json())
else:
    raise ValueError("Data unavailable...")

In [ ]:
grid_options = {
    'enableSorting': True,
    'enableFilter': True,
    'enableColResize': True,
    'enableRangeSelection': True,
    'rowSelection': 'multiple',
    'defaultColDef': {
        'resizable': 'true',
        'suppressRowClickSelection': 'true'
    },
    'columnDefs': [
        {
            'headerName': "Download", 
            'field': "RasterID", 
            'checkboxSelection': True,
            'sortable': False,
            'width': 30,
            'valueFormatter': "function(params) { return ''; }"
        },
        {
            'headerName': "Filename",
            'field': "filename",
            'floatingFilter': True
        }
    ]
}

table_opts = {
    'quick_filter': True,
}

In [ ]:
deselect_action = """
gridOptions.api.deselectAll();
"""

download_action = """
let selectedNodes = gridOptions.api.getSelectedRows();
let x = selectedNodes.map(row => {return row.RasterID});
if (x.length == 0) {
 // nothing selected so do nothing
 return
}
console.log("Downloading rasters...");
console.log(x);

let d_url = 'http://localhost:9090/api/v1/hazard/raster/download/'+x.toString();

var d_link = document.createElement("a");
d_link.href = d_url;
d_link.sandbox = "allow-downloads";
d_link.setAttribute("download", "rdl_hazard_download.zip");
d_link.target = "_parent";

document.body.appendChild(d_link);
d_link.click();
document.body.removeChild(d_link);
"""

buttons = [
    {
        'name': 'Deselect All',
        'action': deselect_action
    },
    {
        'name': 'Download Selected',
         'action': download_action
    },
]

In [ ]:
iactive_df = Grid(
    grid_data=available_rasters, 
    grid_options=grid_options,
    menu={'buttons':buttons},
    **table_opts
)

In [ ]:
# gs = AppLayout(header=m,
#           left_sidebar=m,
#           center=None,
#           right_sidebar=None,
#           footer=iactive_df)
# gs

grid = GridspecLayout(20, 10)
# fill it in with widgets
grid[1:, 0:3] = m
grid[1:, 3:] = iactive_df

# set the widget properties
grid[1:, 0].layout.height = '100%'
# grid[5, 1].layout.height = 'auto'

display(grid)